In [18]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np

In [19]:
#data = pd.read_csv('pmdata.csv')

data = pd.read_csv('PRSA_data_2010.1.1-2014.12.31_1.csv', header=0)


In [20]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [21]:

data = data.dropna()
# data = pd.get_dummies(data)

In [22]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [23]:
wind_direction_mapping = {
    'NW': 1,
    'NE': 2,
    'SE': 3,
    'cv': 4
}

# Replace the string categories with the numeric values
data['cbwd'] = data['cbwd'].replace(wind_direction_mapping)

C:\Users\lyxzz\AppData\Local\Temp\ipykernel_13864\3534025218.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cbwd'] = data['cbwd'].replace(wind_direction_mapping)


In [24]:
train_data_by_month, test_data_by_month = {}, {}
train_data_by_year, test_data_by_year = {}, {}

# Splitting data for each month (month-year combination) separately
for year in data['year'].unique():
    for month in data['month'].unique():
        month_year_data = data[(data['year'] == year) & (data['month'] == month)]
        train_data_by_month[(year, month)], test_data_by_month[(year, month)] = train_test_split(month_year_data,
                                                                                                 test_size=0.2, random_state=12345)

# Combine the monthly splits into yearly splits
for year in data['year'].unique():
    yearly_train_data = pd.concat([train_data_by_month[(year, month)] for month in data['month'].unique()])
    yearly_test_data = pd.concat([test_data_by_month[(year, month)] for month in data['month'].unique()])
    train_data_by_year[year] = yearly_train_data
    test_data_by_year[year] = yearly_test_data

# Combine all monthly splits into a single dataset for full range modeling
train_data_full = pd.concat(train_data_by_month.values())
test_data_full = pd.concat(test_data_by_month.values())

In [30]:

train_data_full.drop(col_drop, axis=1)

,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
620,-11,-5.0,1020.0,2,0.89,0,0
367,-15,-11.0,1034.0,1,4.47,0,0
627,-11,-4.0,1016.0,1,10.29,0,0
90,-23,-11.0,1032.0,1,165.37,0,0
178,-17,-10.0,1030.0,2,0.89,0,0
...,...,...,...,...,...,...,...
43519,-12,0.0,1023.0,1,22.35,0,0
43211,-20,2.0,1028.0,2,9.83,0,0
43384,-19,4.0,1026.0,3,8.04,0,0
43593,-13,-4.0,1026.0,2,5.37,0,0


In [28]:
test_data_by_year.items()
col_drop = ['pm2.5', 'No','year', 'month','day','hour']


In [32]:
year_training_results = {}
year_testing_results = {}
year_prediction_vectors = pd.DataFrame()
year_model_bw = {}
col_drop = ['pm2.5', 'No','year', 'month','day','hour']


for year, train_data in train_data_by_year.items():
    test_data = test_data_by_year[year]
    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # model detail
    nw_model = sm.nonparametric.KernelReg(endog=y_train, exog=X_train, var_type='cccuccc',
                                      bw='cv_ls',ckertype='gaussian',reg_type='lc')

    
    # Make predictions and calculate metrics
    year_model_bw[year] = nw_model.bw
    train_pred,mar_train = nw_model.fit(X_train)
    test_pred,mar_test = nw_model.fit(X_test)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    # Store and print results
    year_training_results[year] = train_rmse
    year_testing_results[year] = test_rmse
    train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
    year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
    print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")
    print(f"Bandwidth:{nw_model.bw}")

Year: 2010, Training RMSE: 31.86590472378974, Testing RMSE: 53.70900321811793
Bandwidth:[0.83258547 1.1260024  0.73680109 0.40349888 3.68490912 1.36037635
 1.6651294 ]


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)


Year: 2011, Training RMSE: 27.286298561496377, Testing RMSE: 54.01900821146512
Bandwidth:[ 0.61186065  1.18752966  0.63278157  0.48898173  3.82811439 -0.99388449
  2.77617404]


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)


Year: 2012, Training RMSE: 33.67293752806816, Testing RMSE: 58.97561304894383
Bandwidth:[ 2.17013167  0.85839525  0.66106162  0.23730865 -3.12053805  0.4266546
  3.64178135]


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x


Year: 2013, Training RMSE: 42.1118798583072, Testing RMSE: 59.431457669126246
Bandwidth:[-0.72244656  1.20896164 -0.76864142  0.49604348 24.42875187  0.59048355
  0.15194374]


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x


Year: 2014, Training RMSE: 32.530600851093986, Testing RMSE: 61.240390169580834
Bandwidth:[0.70201421 1.21337913 0.69068253 0.40122958 3.51604593 0.55189796
 0.59935502]


In [33]:
year_model_bw

{2010: array([0.83258547, 1.1260024 , 0.73680109, 0.40349888, 3.68490912,
        1.36037635, 1.6651294 ]),
 2011: array([ 0.61186065,  1.18752966,  0.63278157,  0.48898173,  3.82811439,
        -0.99388449,  2.77617404]),
 2012: array([ 2.17013167,  0.85839525,  0.66106162,  0.23730865, -3.12053805,
         0.4266546 ,  3.64178135]),
 2013: array([-0.72244656,  1.20896164, -0.76864142,  0.49604348, 24.42875187,
         0.59048355,  0.15194374]),
 2014: array([0.70201421, 1.21337913, 0.69068253, 0.40122958, 3.51604593,
        0.55189796, 0.59935502])}

In [34]:
year_training_results

{2010: 31.86590472378974,
 2011: 27.286298561496377,
 2012: 33.67293752806816,
 2013: 42.1118798583072,
 2014: 32.530600851093986}

In [35]:
year_testing_results

{2010: 53.70900321811793,
 2011: 54.01900821146512,
 2012: 58.97561304894383,
 2013: 59.431457669126246,
 2014: 61.240390169580834}

In [36]:
year_training_results_df =  pd.DataFrame([year_training_results])
year_training_results_df.to_csv('year_training_results_notime.csv', index=False)

In [37]:
year_testing_results_df =  pd.DataFrame([year_testing_results])
year_testing_results_df.to_csv('year_testing_results_notime.csv', index=False)

In [38]:
year_prediction_vectors
year_prediction_vectors.to_csv('year_prediction_vectors_notime.csv', index=False)

In [39]:
year_model_bw_df  = pd.DataFrame(year_model_bw)
year_model_bw_df.to_csv('year_model_bw_notime.csv', index=False)

In [40]:
year_model_bw

{2010: array([0.83258547, 1.1260024 , 0.73680109, 0.40349888, 3.68490912,
        1.36037635, 1.6651294 ]),
 2011: array([ 0.61186065,  1.18752966,  0.63278157,  0.48898173,  3.82811439,
        -0.99388449,  2.77617404]),
 2012: array([ 2.17013167,  0.85839525,  0.66106162,  0.23730865, -3.12053805,
         0.4266546 ,  3.64178135]),
 2013: array([-0.72244656,  1.20896164, -0.76864142,  0.49604348, 24.42875187,
         0.59048355,  0.15194374]),
 2014: array([0.70201421, 1.21337913, 0.69068253, 0.40122958, 3.51604593,
        0.55189796, 0.59935502])}